In [1]:
import json
import logging
from pathlib import Path

from dotenv import load_dotenv

from src.core.agent import SYSTEM_MESSAGE, TASK, Agent
from src.core.dataloader import XmlLoader
from src.core.transformer import RemovePrefix

In [2]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(funcName)s:%(lineno)d - %(message)s",
    # format="%(message)s",
    level=logging.INFO,
    handlers=[logging.StreamHandler()],
)
loading_status = load_dotenv(verbose=True)
logger = logging.getLogger(__name__)

LLM_CONFIG = {"model": "gpt-4o"}
SCHEMA_PATH = Path() / "data" / "data_schema.json"
DATA_DIR = Path() / "data" / "sample"

In [4]:
rules_str = """
1. Score < 650 THEN Reject
    Rule: If the credit score is less than 650, the applicant is rejected.
2. More than 2 PAN IDs THEN Reject
    Rule: If there are more than two PAN IDs, the applicant is rejected.
3. TotalInquiries > 3 THEN Reject
    Rule: If the number of total inquiries exceeds 3, the applicant is rejected.
4. if any Account of AccountType Excluding Credit Cards
    a. SuitFiledStatus = Yes THEN Reject
    Rule: If any Account has a 'Suit Filed' status as 'Yes', the applicant is rejected.
    b. WriteOffAmount > 0 THEN Reject
    Rule: If any Account has a WriteOffAmount greater than 0, the applicant is rejected.
    c. PastDueAmount > 1,500 and DateReported < 12 months THEN Reject
    Rule: If any Account has a PastDueAmount greater than 1,500, and DateReported is within the last 12 months and the
    PaymentStatus in any of the past 12 months is one of the listed delinquency statuses, the applicant is rejected.
    d. AccountStatus in {WDF, SUB, DBT, etc.} THEN Reject
    Rule: If any account has a status in the listed statuses, the applicant is rejected.
    e. PastDueAmount > 1,500 and DateReported > 12 months THEN Reject
    Rule: If any Account has a PastDueAmount greater than 1,500, DateReported is more than 12 months ago, and
    PaymentStatus in any of the past 6 months indicates delinquency, the applicant is rejected.
5. if any Account of AccountType = Credit Card  and PastDueAmount > 5,000 THEN Reject
    Rule: If any credit card account has a PastDueAmount greater than 5,000, the applicant is rejected.
"""


with open(SCHEMA_PATH, "r") as f:
    json_schema = json.load(f)

data = []
data_id = 1
for file_path in DATA_DIR.glob("*.html"):
    loder = XmlLoader.from_path(file_path)
    d = RemovePrefix().transform(loder.document)
    data.append({"id": data_id, "data": d})
    data_id += 1

In [4]:
# system_message = SYSTEM_MESSAGE.format(RULES=rules_str, JSON_SCHEMA=json_schema)
# print(system_message)

In [5]:
agent = Agent(
    system_message=SYSTEM_MESSAGE.format(RULES=rules_str, JSON_SCHEMA=json_schema),
    llm_config={"model": "gpt-4o"},
    enable_cache=False,
    enable_logger=True,
)

agent.set_data(data)

resp = agent.resolve_task(TASK)

00:17:24 - LiteLLM:INFO: utils.py:2952 - 
LiteLLM completion() model= gpt-4o; provider = openai
2024-10-22 00:17:24,162 - INFO - LiteLLM - _check_valid_arg:2952 - 
LiteLLM completion() model= gpt-4o; provider = openai
2024-10-22 00:17:25,271 - INFO - httpx - _send_single_request:1038 - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
To create the `evaluate_credit_score` function, let's first reflect on the rules and the JSON schema to identify the necessary attributes and their types. This will help us understand what data we need to extract and how to process it.

### Reflection on Rules and Schema

1. **Score < 650 THEN Reject**
   - Attribute: `Score.Value`
   - Type: String (needs to be converted to integer for comparison)

2. **More than 2 PAN IDs THEN Reject**
   - Attribute: `IDAndContactInfo.IdentityInfo.PANId`
   - Type: Object (we need to count the number of PAN IDs)

3. **TotalInquiries > 3 THEN Reject**
   - Attribute: `RecentActivities.Total

AttributeError: 'Executator' object has no attribute 'error'

In [5]:
logger.info(resp.code_content)

In [6]:
logger.info(resp.response_string)

"from datetime import datetime\n\ndef evaluate_credit_score(data):\n    # Rule 1: Score < 650\n    score_value = int(data.get('Score', {}).get('Value', '0'))\n    if score_value < 650:\n        return 'Rejected'\n    \n    # Rule 2: More than 2 PAN IDs\n    pan_id_count = len(data.get('IDAndContactInfo', {}).get('IdentityInfo', {}).get('PANId', []))\n    if pan_id_count > 2:\n        return 'Rejected'\n    \n    # Rule 3: TotalInquiries > 3\n    total_inquiries = int(data.get('RecentActivities', {}).get('TotalInquiries', '0'))\n    if total_inquiries > 3:\n        return 'Rejected'\n    \n    # Rule 4: Account Rules\n    accounts = data.get('AccountDetails', {}).get('Account', [])\n    for account in accounts:\n        account_type = account.get('AccountType', '')\n        suit_filed_status = account.get('SuitFiledStatus', 'No')\n        past_due_amount = int(account.get('PastDueAmount', '0'))\n        date_reported_str = account.get('DateReported', '')\n        date_reported = datetim

In [6]:
{"version": "1.0", "rules_str": rules_str}

{'version': '1.0',
 'rules_str': "\n1. Score < 650 THEN Reject\n    Rule: If the credit score is less than 650, the applicant is rejected.\n2. More than 2 PAN IDs THEN Reject\n    Rule: If there are more than two PAN IDs, the applicant is rejected.\n3. TotalInquiries > 3 THEN Reject\n    Rule: If the number of total inquiries exceeds 3, the applicant is rejected.\n4. if any Account of AccountType Excluding Credit Cards\n    a. SuitFiledStatus = Yes THEN Reject\n    Rule: If any Account has a 'Suit Filed' status as 'Yes', the applicant is rejected.\n    b. WriteOffAmount > 0 THEN Reject\n    Rule: If any Account has a WriteOffAmount greater than 0, the applicant is rejected.\n    c. PastDueAmount > 1,500 and DateReported < 12 months THEN Reject\n    Rule: If any Account has a PastDueAmount greater than 1,500, and DateReported is within the last 12 months and the\n    PaymentStatus in any of the past 12 months is one of the listed delinquency statuses, the applicant is rejected.\n    d. 

In [ ]:
{
    "version": "1.0",
    "rules_str": "\n1. Score < 650 THEN Reject\n    Rule: If the credit score is less than 650, the applicant is rejected.\n2. More than 2 PAN IDs THEN Reject\n    Rule: If there are more than two PAN IDs, the applicant is rejected.\n3. TotalInquiries > 3 THEN Reject\n    Rule: If the number of total inquiries exceeds 3, the applicant is rejected.\n4. if any Account of AccountType Excluding Credit Cards\n    a. SuitFiledStatus = Yes THEN Reject\n    Rule: If any Account has a 'Suit Filed' status as 'Yes', the applicant is rejected.\n    b. WriteOffAmount > 0 THEN Reject\n    Rule: If any Account has a WriteOffAmount greater than 0, the applicant is rejected.\n    c. PastDueAmount > 1,500 and DateReported < 12 months THEN Reject\n    Rule: If any Account has a PastDueAmount greater than 1,500, and DateReported is within the last 12 months and the\n    PaymentStatus in any of the past 12 months is one of the listed delinquency statuses, the applicant is rejected.\n    d. AccountStatus in {WDF, SUB, DBT, etc.} THEN Reject\n    Rule: If any account has a status in the listed statuses, the applicant is rejected.\n    e. PastDueAmount > 1,500 and DateReported > 12 months THEN Reject\n    Rule: If any Account has a PastDueAmount greater than 1,500, DateReported is more than 12 months ago, and\n    PaymentStatus in any of the past 6 months indicates delinquency, the applicant is rejected.\n5. if any Account of AccountType = Credit Card  and PastDueAmount > 5,000 THEN Reject\n    Rule: If any credit card account has a PastDueAmount greater than 5,000, the applicant is rejected.\n",
}